<a href="https://colab.research.google.com/github/holly-hewitt/Abnormal-Infant-Movement-Detection/blob/main/Model%20Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle

# Load the data from pickle
with open('/content/drive/MyDrive/Abnormal Infant Movement Pickles/padded_pca_accelX.pickle', 'rb') as f:
    padded_pca_accelX = pickle.load(f)

with open('/content/drive/MyDrive/Abnormal Infant Movement Pickles/accelAbnormalY.pickle', 'rb') as f:
    accelAbnormalY = pickle.load(f)

In [2]:
from tensorflow.compat.v1.keras.utils import to_categorical

# One hot encoding of Y_train and Y_test


Y_encoded = to_categorical(accelAbnormalY, num_classes=3)


In [3]:
# Create test and train set
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_pca_accelX, Y_encoded, test_size=0.2, random_state=42)

# Sanity check: Print results
print("X_train:")
print(len(X_train))

print("Y train:")
print(len(y_train))

print("Y train shape:")
print(y_train.shape)

X_train:
128
Y train:
128
Y train shape:
(128, 3)


In [4]:
import numpy as np

X_train = np.array(X_train)
X_train = X_train.reshape((X_train.shape[0], 1842, 100, 3))
print(X_train.shape)
print(type(X_train))

X_test = np.array(X_test)
X_test = X_test.reshape((X_test.shape[0], 1842, 100, 3))
print(X_test.shape)



(128, 1842, 100, 3)
<class 'numpy.ndarray'>
(33, 1842, 100, 3)


In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking

model = Sequential()
model.add(Masking(mask_value=0., input_shape=(1842, 100, 3)))
model.add(LSTM(units=64, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print("Accuracy:", accuracy)

tf.keras.backend.clear_session()


ValueError: Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 1842, 100, 3)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
import tensorflow as tf

# Simplified model
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(1842, 100, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(1842, 100, 3)))  # Reduced filters
model.add(MaxPooling2D((2, 2), strides=2))  # Increased stride in pooling layer
model.add(Conv2D(32, (3, 3), activation='relu'))  # Reduced filters
model.add(MaxPooling2D((2, 2), strides=2))  # Increased stride in pooling layer
model.add(Flatten())
model.add(Dense(32, activation='relu'))  # Reduced size of the dense layer
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss=categorical_crossentropy, metrics=['accuracy'])

# Assuming X_train, y_train, X_test, and y_test are properly defined and preprocessed
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print("Accuracy:", accuracy)

tf.keras.backend.clear_session()



Epoch 1/10
4/4 [==============================] - 2s 309ms/step - loss: 4.3240 - accuracy: 0.4062 - val_loss: 1.2309 - val_accuracy: 0.6667
Epoch 2/10
4/4 [==============================] - 1s 151ms/step - loss: 3.4092 - accuracy: 0.5781 - val_loss: 1.7985 - val_accuracy: 0.4242
Epoch 3/10
4/4 [==============================] - 1s 169ms/step - loss: 1.8512 - accuracy: 0.6562 - val_loss: 1.1568 - val_accuracy: 0.6061
Epoch 4/10
4/4 [==============================] - 1s 151ms/step - loss: 0.8452 - accuracy: 0.7578 - val_loss: 1.9592 - val_accuracy: 0.4242
Epoch 5/10
4/4 [==============================] - 1s 171ms/step - loss: 0.6539 - accuracy: 0.7344 - val_loss: 0.7782 - val_accuracy: 0.6364
Epoch 6/10
4/4 [==============================] - 1s 150ms/step - loss: 0.6722 - accuracy: 0.6406 - val_loss: 0.8309 - val_accuracy: 0.6061
Epoch 7/10
4/4 [==============================] - 1s 166ms/step - loss: 0.4891 - accuracy: 0.7500 - val_loss: 1.1140 - val_accuracy: 0.4242
Epoch 8/10
4/4 [====

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    # Assuming input_shape is correctly set for your data
    Conv2D(64, (3, 3), activation='relu', input_shape=(1842, 100, 3), kernel_initializer='he_uniform'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu', kernel_initializer='he_uniform'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Add callbacks as needed
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

# Train the model with callbacks
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=callbacks)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print("Accuracy:", accuracy)

tf.keras.backend.clear_session()


Epoch 1/10
4/4 [==============================] - ETA: 0s - loss: 63.0062 - accuracy: 0.5547

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 14s 3s/step - loss: 63.0062 - accuracy: 0.5547 - val_loss: 9.9123 - val_accuracy: 0.5758
Epoch 2/10
4/4 [==============================] - 6s 2s/step - loss: 22.3765 - accuracy: 0.4062 - val_loss: 6.9560 - val_accuracy: 0.4242
Epoch 3/10
4/4 [==============================] - 1s 287ms/step - loss: 11.0674 - accuracy: 0.6406 - val_loss: 15.4207 - val_accuracy: 0.4242
Epoch 4/10
4/4 [==============================] - 11s 4s/step - loss: 5.6049 - accuracy: 0.7031 - val_loss: 3.8462 - val_accuracy: 0.5152
Epoch 5/10
4/4 [==============================] - 6s 2s/step - loss: 1.8306 - accuracy: 0.6562 - val_loss: 0.9936 - val_accuracy: 0.6970
Epoch 6/10
4/4 [==============================] - 6s 2s/step - loss: 1.0908 - accuracy: 0.6172 - val_loss: 0.9303 - val_accuracy: 0.3939
Epoch 7/10
4/4 [==============================] - 5s 2s/step - loss: 0.7810 - accuracy: 0.6797 - val_loss: 0.9222 - val_accuracy: 0.3939
Epoch 8/10
4/4 [===========================

In [7]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from kerastuner import RandomSearch

def build_model(hp):
    model = Sequential([
        Conv2D(
            filters=hp.Int('conv_1_filters', min_value=32, max_value=128, step=32),
            kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
            activation='relu',
            input_shape=(1842, 100, 3)
        ),
        MaxPooling2D((2, 2)),
        Conv2D(
            filters=hp.Int('conv_2_filters', min_value=32, max_value=128, step=32),
            kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
            activation='relu'
        ),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(
            units=hp.Int('dense_units', min_value=32, max_value=128, step=32),
            activation='relu'
        ),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(
            hp.Float(
                'learning_rate',
                min_value=1e-4,
                max_value=1e-2,
                sampling='LOG'
            )
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Set the number of trials, or experiments, to conduct.
    executions_per_trial=1,  # Set the number of models that should be built and fit for each trial for more reliable results.
    directory='my_dir',  # Directory where the search results will be stored.
    project_name='keras_tuner_demo'
)

tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('dense_units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print("Accuracy:", accuracy)

tf.keras.backend.clear_session()


<ipython-input-8-6056af37e857>:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
128               |128               |conv_1_filters
5                 |5                 |conv_1_kernel
128               |128               |conv_2_filters
5                 |5                 |conv_2_kernel
32                |32                |dense_units
0.0039792         |0.0039792         |learning_rate

Epoch 1/10


In [18]:
tf.keras.backend.clear_session()


In [19]:
def build_model(hp):
    model = Sequential([
        Conv2D(
            filters=hp.Int('conv_1_filters', min_value=32, max_value=64, step=32),  # Reduced upper limit
            kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
            activation='relu',
            input_shape=(1842, 100, 3)
        ),
        MaxPooling2D((2, 2)),
        Conv2D(
            filters=hp.Int('conv_2_filters', min_value=32, max_value=64, step=32),  # Reduced upper limit
            kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
            activation='relu'
        ),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(
            units=hp.Int('dense_units', min_value=32, max_value=64, step=32),  # Reduced upper limit
            activation='relu'
        ),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])

    # Adjusted learning rate search space to prevent very high values
    model.compile(
        optimizer=Adam(
            hp.Float(
                'learning_rate',
                min_value=1e-4,
                max_value=1e-3,
                sampling='LOG'
            )
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Set the number of trials, or experiments, to conduct.
    executions_per_trial=1,  # Set the number of models that should be built and fit for each trial for more reliable results.
    directory='my_dir',  # Directory where the search results will be stored.
    project_name='keras_tuner_demo'
)

# Consider using a smaller initial batch size if memory issues persist
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=10)  # Reduced batch size

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('dense_units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print("Accuracy:", accuracy)

tf.keras.backend.clear_session()


Trial 10 Complete [00h 00m 17s]

Best val_accuracy So Far: 0.7575757503509521
Total elapsed time: 00h 10m 35s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 32 and the optimal learning rate for the optimizer
is 0.00018705716077490754.

Epoch 1/50


ResourceExhaustedError: Graph execution error:

Detected at node sequential_1/conv2d_3/Relu defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy

# Simplified model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(1842, 100, 3)))  # Reduced filters
model.add(MaxPooling2D((2, 2), strides=2))  # Increased stride in pooling layer
model.add(Conv2D(32, (3, 3), activation='relu'))  # Reduced filters
model.add(MaxPooling2D((2, 2), strides=2))  # Increased stride in pooling layer
model.add(Flatten())
model.add(Dense(32, activation='relu'))  # Reduced size of the dense layer
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss=categorical_crossentropy, metrics=['accuracy'])

# Assuming X_train, y_train, X_test, and y_test are properly defined and preprocessed
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print("Accuracy:", accuracy)

tf.keras.backend.clear_session()



Epoch 1/10
4/4 [==============================] - 2s 308ms/step - loss: 4.2179 - accuracy: 0.6250 - val_loss: 0.7244 - val_accuracy: 0.5455
Epoch 2/10
4/4 [==============================] - 1s 172ms/step - loss: 1.6621 - accuracy: 0.5000 - val_loss: 0.8907 - val_accuracy: 0.3939
Epoch 3/10
4/4 [==============================] - 1s 165ms/step - loss: 0.7388 - accuracy: 0.6875 - val_loss: 0.8964 - val_accuracy: 0.3939
Epoch 4/10
4/4 [==============================] - 1s 145ms/step - loss: 0.7625 - accuracy: 0.6562 - val_loss: 0.7961 - val_accuracy: 0.3939
Epoch 5/10
4/4 [==============================] - 1s 176ms/step - loss: 0.6843 - accuracy: 0.6875 - val_loss: 0.8533 - val_accuracy: 0.3939
Epoch 6/10
4/4 [==============================] - 1s 162ms/step - loss: 0.6509 - accuracy: 0.6719 - val_loss: 0.7689 - val_accuracy: 0.4545
Epoch 7/10
4/4 [==============================] - 1s 167ms/step - loss: 0.5784 - accuracy: 0.7344 - val_loss: 0.8031 - val_accuracy: 0.3939
Epoch 8/10
4/4 [====

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np
import math

model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(1842, 100, 3), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(3, activation='softmax')
])
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss=categorical_crossentropy, metrics=['accuracy'])

# Assuming X_train, y_train, X_test, and y_test are properly defined and preprocessed
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print("Accuracy:", accuracy)


Epoch 1/10
4/4 [==============================] - 154s 37s/step - loss: 1.9492 - accuracy: 0.4375 - val_loss: 1.7019 - val_accuracy: 0.6667
Epoch 2/10
3/4 [=====================>........] - ETA: 33s - loss: 1.5488 - accuracy: 0.4688 